In [27]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import linprog

%run func_library.ipynb

### Attribute Information:

1. variance of Wavelet Transformed image (continuous) 
2. skewness of Wavelet Transformed image (continuous) 
3. curtosis of Wavelet Transformed image (continuous) 
4. entropy of image (continuous) 
5. class (integer) 

In [28]:
data = pd.read_csv('datasets/data_banknote_authentication.txt',header=None)

In [29]:
data.columns = ['variance', 'skewness', 'curtosis', 'entropy', 'class']

In [30]:
data['class'].value_counts()

0    762
1    610
Name: class, dtype: int64

### Getting list of all functions from the Function Library

In [31]:
funcObj = FunctionLibrary(1) # Declaring a default func object
list_of_functions = [func for func in dir(funcObj) if callable(getattr(funcObj, func)) and not func.startswith("__")]
list_of_functions

['Normx', 'cosx', 'cube', 'expsquare', 'expx', 'sinx', 'subsquare', 'x2_x3']

In [32]:
def transform(x):
    ip = FunctionLibrary(x)
    return getattr(ip, func)()

### Iterating over all transformations and checking for linear separability using LP

In [35]:
for func in list_of_functions:

    print('Transforming using',func,'as the transformation function')
    data_copy = data.copy()
    
    data_copy.iloc[:,0:4] = data_copy.iloc[:,0:4].applymap(lambda x: transform(x))
 
    dic = {0: '0', 1: '1'}

    for i in dic.keys():
        data_copy["newTarget"] = np.where(data_copy['class'] == i, 1 , -1)

        from sklearn.preprocessing import StandardScaler
        sc = StandardScaler()
        tmp = data_copy.iloc[:,[1,3]].values
        tmp = sc.fit_transform(tmp)

        xx = np.array(data_copy.newTarget.values.reshape(-1,1) * tmp)
        t = np.where(data_copy['class'] == i, 1 , -1)

        #2-D array which, when matrix-multiplied by x, gives the values of 
        #the upper-bound inequality constraints at x.
        A_ub = np.append(xx, t.reshape(-1,1), 1)

        #1-D array of values representing the upper-bound of each 
        #inequality constraint (row) in A_ub.
        b_ub = np.repeat(-1, A_ub.shape[0]).reshape(-1,1)

        # Coefficients of the linear objective function to be minimized.
        c_obj = np.repeat(1, A_ub.shape[1])
        res = linprog(c=c_obj, A_ub=A_ub, b_ub=b_ub,
                      options={"disp": False})

        if res.success:
            print('There is linear separability')
        else:
            print('No linear separability')
        break #remove break in case of multiclass classification 
    print('-'*80)

Transforming using Normx as the transformation function
No linear separability
--------------------------------------------------------------------------------
Transforming using cosx as the transformation function
No linear separability
--------------------------------------------------------------------------------
Transforming using cube as the transformation function
No linear separability
--------------------------------------------------------------------------------
Transforming using expsquare as the transformation function


/Users/ambuj/Documents/MS Stuff/ml_cs_6350/ML_Project/env_ml/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1545: RuntimeWarning: overflow encountered in multiply
  sqr = np.multiply(arr, arr, out=arr)
/Users/ambuj/Documents/MS Stuff/ml_cs_6350/ML_Project/env_ml/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


No linear separability
--------------------------------------------------------------------------------
Transforming using expx as the transformation function
No linear separability
--------------------------------------------------------------------------------
Transforming using sinx as the transformation function
No linear separability
--------------------------------------------------------------------------------
Transforming using subsquare as the transformation function
No linear separability
--------------------------------------------------------------------------------
Transforming using x2_x3 as the transformation function
No linear separability
--------------------------------------------------------------------------------
